In [1]:
# medical-diagnosis-system/
# │── data/                      # X-ray images & text datasets
# │── models/                    # Trained CNN and NLP models
# │── src/
# │   ├── cnn_model.py            # CNN Model for X-ray classification
# │   ├── nlp_model.py            # NLP Model for medical text classification
# │   ├── api.py                  # Flask API for inference
# │── app/
# │   ├── frontend.py              # Streamlit frontend (optional)
# │── requirements.txt             # Dependencies
# │── Dockerfile                   # Docker configuration
# │── main.py                      # Entry point


In [2]:
#  ! pip install tensorflow keras opencv-python nltk transformers      

#  jupyter nbconvert --to script AI-Based_Medical_Diagnosis_System.ipynb

# streamlit run AI-Based_Medical_Diagnosis_System.py 

In [3]:

################# 14thh

# ✅ All-in-One AI Medical Diagnosis (Multimodal Fusion + X-ray + Report + Multiclass + EDA + Export + DB Ready)

import streamlit as st
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet18
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from sklearn.model_selection import train_test_split
from fpdf import FPDF
from datetime import datetime
from pymongo import MongoClient

# ====================== 🌈 Streamlit UI Setup ====================== #
st.set_page_config(page_title="🧠 AI Medical Diagnosis", layout="wide")

# 🌈 Theme and custom UI
st.markdown("""
    <style>
        .stApp {
            background-color: #0a192f;
            color: white;
            padding: 20px;
            border-radius: 15px;
        }
        .css-18e3th9, .css-1d391kg {
            background-color: #112240 !important;
            color: white !important;
            border-radius: 10px;
        }
        .stTextInput>div>div>input, .stTextArea textarea {
            color: white !important;
            background-color: #233554 !important;
        }
        .block-container {
            padding-top: 2rem;
        }
    </style>
""", unsafe_allow_html=True)

st.title("🩺 Advanced AI Medical Diagnosis (X-ray + Report + Fusion + Multiclass)")

# MongoDB connection
client = MongoClient("mongodb://localhost:27017")
db = client["MedicalAI"]
collection = db["predictions"]

# =========================== 📥 Upload CSV and Show Preview ============================= #
uploaded_csv = st.file_uploader("📄 Upload Radiology Report CSV", type="csv")
tfidf, clf = None, None
patient_ids = []

if uploaded_csv:
    df = pd.read_csv(uploaded_csv)
    df['patient_id'] = ['PID' + str(random.randint(10000, 99999)) for _ in range(len(df))]
    df['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    df['age'] = np.random.randint(25, 80, size=len(df))
    df['sex'] = np.random.choice(['Male', 'Female'], size=len(df))
    df['symptoms'] = np.random.choice([
        'Cough, Fever', 'Chest Pain, Breathlessness', 'Fatigue, Cough', 'Shortness of Breath', 'No major symptoms'
    ], size=len(df))

    st.subheader("📊 Sample Data")
    st.dataframe(df.head(), use_container_width=True)

    st.subheader("📈 Label Distribution")
    fig, ax = plt.subplots(figsize=(4, 2.5))
    df['label'].value_counts().plot(kind='bar', ax=ax, color=sns.color_palette("Set2"))
    ax.set_xlabel("Label", color='white')
    ax.set_ylabel("Count", color='white')
    ax.tick_params(colors='white')
    fig.patch.set_facecolor('#112240')
    ax.set_facecolor('#112240')
    st.pyplot(fig)

    # ======================== 📊 EDA ============================= #
    st.subheader("📌 Dataset Overview")
    st.dataframe(df.describe(include='all'))

    # =========================== 🧠 NLP Model ============================= #
    st.subheader("🧠 Report Classification (TF-IDF + Logistic Regression)")
    tfidf = TfidfVectorizer(max_features=300)
    X = tfidf.fit_transform(df['report_text'])
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    probs = clf.predict_proba(X_test)
    acc = accuracy_score(y_test, y_pred)
    st.write("✅ Accuracy:", round(acc * 100, 2), "%")

    st.subheader("📉 Confusion Matrix")
    fig_cm, ax_cm = plt.subplots(figsize=(4, 2.5))
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='coolwarm', ax=ax_cm, cbar=False)
    ax_cm.set_xlabel("Predicted", color='white')
    ax_cm.set_ylabel("Actual", color='white')
    ax_cm.tick_params(colors='white')
    fig_cm.patch.set_facecolor('#112240')
    ax_cm.set_facecolor('#112240')
    st.pyplot(fig_cm)

    st.subheader("🧾 Classification Report")
    st.dataframe(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

    # =========================== 📊 Live Dashboard ============================= #
    st.subheader("📈 Live Confidence Score Dashboard")
    label_names = clf.classes_
    class_probs = clf.predict_proba(X_test[:5])
    for i in range(min(5, len(class_probs))):
        st.markdown(f"**Patient {i+1}:**")
        chart_data = pd.DataFrame({
            'Label': label_names,
            'Probability': class_probs[i]
        })
        st.bar_chart(chart_data.set_index('Label'))

# =========================== 🖼️ CNN Model for X-ray Image ============================= #
st.header("🩻 X-ray Diagnosis (CNN Binary)")

@st.cache_resource
def load_cnn_model():
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 2)
    return model

cnn_model = load_cnn_model()
cnn_model.eval()

image_file = st.file_uploader("🖼️ Upload Chest X-ray Image", type=['png', 'jpg', 'jpeg'])
pred_label = None
if image_file:
    image = Image.open(image_file).convert('RGB')
    st.image(image, caption="Uploaded Chest X-ray", width=250)

    transform = transforms.Compose([
        transforms.Resize((124, 224)),
        transforms.ToTensor(),
    ])
    input_tensor = transform(image).unsqueeze(0)
    output = cnn_model(input_tensor)
    pred_label = torch.argmax(output, 1).item()
    st.success("✅ CNN Prediction: " + ("Normal" if pred_label == 0 else "⚠️ Abnormal"))

# =========================== 🔀 Fusion Model ============================= #
st.header("🔬 Multimodal Prediction (Fusion: X-ray + Report)")

user_text = st.text_area("✍️ Enter a Radiology Report")
if st.button("🔍 Predict from Report + X-ray"):
    if user_text.strip() != "" and tfidf and clf:
        text_pred = clf.predict(tfidf.transform([user_text]))[0]
        prob = clf.predict_proba(tfidf.transform([user_text]))[0]
        confidence = round(np.max(prob) * 100, 2)
        fusion_result = f"🧠 NLP: {text_pred} ({confidence}%)"
        if pred_label is not None:
            fusion_result += f" | 🩻 CNN: {'Normal' if pred_label == 0 else 'Abnormal'}"
        st.success("✅ Combined Prediction: " + fusion_result)

        st.progress(int(confidence))

        # 🔁 Store in MongoDB
        fusion_data = {
            "report": user_text,
            "cnn_label": "Normal" if pred_label == 0 else "Abnormal",
            "nlp_label": text_pred,
            "confidence": confidence,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
       ## collection.insert_one(fusion_data)
       ## st.info("📥 Prediction saved to MongoDB.")
    else:
        st.warning("⚠️ Please upload X-ray + enter report")

# =========================== 📤 Export Predictions ============================= #
if uploaded_csv and st.button("📥 Export Predictions as CSV + PDF"):
    df['predicted_label'] = clf.predict(tfidf.transform(df['report_text']))
    df.to_csv("predicted_multimodal.csv", index=False)
    st.success("✅ Exported as 'predicted_multimodal.csv'")

    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    for i, row in df.iterrows():
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.cell(200, 10, txt=f"Patient ID: {row['patient_id']}", ln=True)
        pdf.cell(200, 10, txt=f"Timestamp: {row['timestamp']}", ln=True)
        pdf.cell(200, 10, txt=f"Age: {row['age']} | Sex: {row['sex']}", ln=True)
        pdf.cell(200, 10, txt=f"Symptoms: {row['symptoms']}", ln=True)
        pdf.cell(200, 10, txt=f"Report: {row['report_text'][:100]}...", ln=True)
        pdf.cell(200, 10, txt=f"Predicted Diagnosis: {row['predicted_label']}", ln=True)
    pdf.output("predicted_reports.pdf")
    st.success("📄 PDF Exported as 'predicted_reports.pdf'")




2025-04-10 14:24:28.420 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 14:24:28.420 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 14:24:28.907 
  command:

    streamlit run C:\Python312\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-10 14:24:28.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 14:24:28.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 14:24:28.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 14:24:28.970 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare 

In [4]:
# ####################### 13th

# # ✅ All-in-One AI Medical Diagnosis (Multimodal Fusion + X-ray + Report + Multiclass + EDA + Export + DB Ready)

# import streamlit as st
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torchvision.transforms as transforms
# from torchvision.models import resnet18
# from PIL import Image
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# import random
# from sklearn.model_selection import train_test_split
# from fpdf import FPDF
# from datetime import datetime
# from pymongo import MongoClient

# # ====================== 🌈 Streamlit UI Setup ====================== #
# st.set_page_config(page_title="🧠 AI Medical Diagnosis", layout="wide")

# # 🌈 Theme and custom UI
# st.markdown("""
#     <style>
#         .stApp {
#             background-color: #0a192f;
#             color: white;
#             padding: 20px;
#             border-radius: 15px;
#         }
#         .css-18e3th9, .css-1d391kg {
#             background-color: #112240 !important;
#             color: white !important;
#             border-radius: 10px;
#         }
#         .stTextInput>div>div>input, .stTextArea textarea {
#             color: white !important;
#             background-color: #233554 !important;
#         }
#         .block-container {
#             padding-top: 2rem;
#         }
#     </style>
# """, unsafe_allow_html=True)

# st.title("🩺 Advanced AI Medical Diagnosis (X-ray + Report + Fusion + Multiclass)")

# # MongoDB connection
# client = MongoClient("mongodb://localhost:27017")
# db = client["MedicalAI"]
# collection = db["predictions"]

# # =========================== 📥 Upload CSV and Show Preview ============================= #
# uploaded_csv = st.file_uploader("📄 Upload Radiology Report CSV", type="csv")
# tfidf, clf = None, None
# patient_ids = []

# if uploaded_csv:
#     df = pd.read_csv(uploaded_csv)
#     df['patient_id'] = ['PID' + str(random.randint(10000, 99999)) for _ in range(len(df))]
#     df['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#     df['age'] = np.random.randint(25, 80, size=len(df))
#     df['sex'] = np.random.choice(['Male', 'Female'], size=len(df))
#     df['symptoms'] = np.random.choice([
#         'Cough, Fever', 'Chest Pain, Breathlessness', 'Fatigue, Cough', 'Shortness of Breath', 'No major symptoms'
#     ], size=len(df))

#     st.subheader("📊 Sample Data")
#     st.dataframe(df.head(), use_container_width=True)

#     st.subheader("📈 Label Distribution")
#     fig, ax = plt.subplots(figsize=(4, 2.5))
#     df['label'].value_counts().plot(kind='bar', ax=ax, color=sns.color_palette("Set2"))
#     ax.set_xlabel("Label", color='white')
#     ax.set_ylabel("Count", color='white')
#     ax.tick_params(colors='white')
#     fig.patch.set_facecolor('#112240')
#     ax.set_facecolor('#112240')
#     st.pyplot(fig)

#     # ======================== 📊 EDA ============================= #
#     st.subheader("📌 Dataset Overview")
#     st.dataframe(df.describe(include='all'))

#     # =========================== 🧠 NLP Model ============================= #
#     st.subheader("🧠 Report Classification (TF-IDF + Logistic Regression)")
#     tfidf = TfidfVectorizer(max_features=300)
#     X = tfidf.fit_transform(df['report_text'])
#     y = df['label']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     clf = LogisticRegression(max_iter=1000)
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     probs = clf.predict_proba(X_test)
#     acc = accuracy_score(y_test, y_pred)
#     st.write("✅ Accuracy:", round(acc * 100, 2), "%")

#     st.subheader("📉 Confusion Matrix")
#     fig_cm, ax_cm = plt.subplots(figsize=(4, 2.5))
#     sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='coolwarm', ax=ax_cm, cbar=False)
#     ax_cm.set_xlabel("Predicted", color='white')
#     ax_cm.set_ylabel("Actual", color='white')
#     ax_cm.tick_params(colors='white')
#     fig_cm.patch.set_facecolor('#112240')
#     ax_cm.set_facecolor('#112240')
#     st.pyplot(fig_cm)

#     st.subheader("🧾 Classification Report")
#     st.dataframe(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

#     # =========================== 📊 Live Dashboard ============================= #
#     st.subheader("📈 Live Confidence Score Dashboard")
#     label_names = clf.classes_
#     class_probs = clf.predict_proba(X_test[:5])
#     for i in range(5):
#         st.markdown(f"**Patient {i+1}:**")
#         chart_data = pd.DataFrame({
#             'Label': label_names,
#             'Probability': class_probs[i]
#         })
#         st.bar_chart(chart_data.set_index('Label'))

# # =========================== 🖼️ CNN Model for X-ray Image ============================= #
# st.header("🩻 X-ray Diagnosis (CNN Binary)")

# @st.cache_resource
# def load_cnn_model():
#     model = resnet18(pretrained=True)
#     model.fc = nn.Linear(model.fc.in_features, 2)
#     return model

# cnn_model = load_cnn_model()
# cnn_model.eval()

# image_file = st.file_uploader("🖼️ Upload Chest X-ray Image", type=['png', 'jpg', 'jpeg'])
# pred_label = None
# if image_file:
#     image = Image.open(image_file).convert('RGB')
#     st.image(image, caption="Uploaded Chest X-ray", width=250)

#     transform = transforms.Compose([
#         transforms.Resize((124, 224)),
#         transforms.ToTensor(),
#     ])
#     input_tensor = transform(image).unsqueeze(0)
#     output = cnn_model(input_tensor)
#     pred_label = torch.argmax(output, 1).item()
#     st.success("✅ CNN Prediction: " + ("Normal" if pred_label == 0 else "⚠️ Abnormal"))

# # =========================== 🔀 Fusion Model ============================= #
# st.header("🔬 Multimodal Prediction (Fusion: X-ray + Report)")

# user_text = st.text_area("✍️ Enter a Radiology Report")
# if st.button("🔍 Predict from Report + X-ray"):
#     if user_text.strip() != "" and tfidf and clf:
#         text_pred = clf.predict(tfidf.transform([user_text]))[0]
#         prob = clf.predict_proba(tfidf.transform([user_text]))[0]
#         confidence = round(np.max(prob) * 100, 2)
#         fusion_result = f"🧠 NLP: {text_pred} ({confidence}%)"
#         if pred_label is not None:
#             fusion_result += f" | 🩻 CNN: {'Normal' if pred_label == 0 else 'Abnormal'}"
#         st.success("✅ Combined Prediction: " + fusion_result)

#         st.progress(int(confidence))

#         # 🔁 Store in MongoDB
#         fusion_data = {
#             "report": user_text,
#             "cnn_label": "Normal" if pred_label == 0 else "Abnormal",
#             "nlp_label": text_pred,
#             "confidence": confidence,
#             "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#         }
#         collection.insert_one(fusion_data)
#         st.info("📥 Prediction saved to MongoDB.")
#     else:
#         st.warning("⚠️ Please upload X-ray + enter report")

# # =========================== 📤 Export Predictions ============================= #
# if uploaded_csv and st.button("📥 Export Predictions as CSV + PDF"):
#     df['predicted_label'] = clf.predict(tfidf.transform(df['report_text']))
#     df.to_csv("predicted_multimodal.csv", index=False)
#     st.success("✅ Exported as 'predicted_multimodal.csv'")

#     pdf = FPDF()
#     pdf.set_auto_page_break(auto=True, margin=15)
#     for i, row in df.iterrows():
#         pdf.add_page()
#         pdf.set_font("Arial", size=12)
#         pdf.cell(200, 10, txt=f"Patient ID: {row['patient_id']}", ln=True)
#         pdf.cell(200, 10, txt=f"Timestamp: {row['timestamp']}", ln=True)
#         pdf.cell(200, 10, txt=f"Age: {row['age']} | Sex: {row['sex']}", ln=True)
#         pdf.cell(200, 10, txt=f"Symptoms: {row['symptoms']}", ln=True)
#         pdf.cell(200, 10, txt=f"Report: {row['report_text'][:100]}...", ln=True)
#         pdf.cell(200, 10, txt=f"Predicted Diagnosis: {row['predicted_label']}", ln=True)
#     pdf.output("predicted_reports.pdf")
#     st.success("📄 PDF Exported as 'predicted_reports.pdf'")




In [5]:
####################### 12thh

# # ✅ All-in-One AI Medical Diagnosis (Multimodal Fusion + X-ray + Report + Multiclass + EDA + Export + DB Ready)

# import streamlit as st
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torchvision.transforms as transforms
# from torchvision.models import resnet18
# from PIL import Image
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# import random
# from sklearn.model_selection import train_test_split
# from fpdf import FPDF
# from datetime import datetime

# # ====================== 🌈 Streamlit UI Setup ====================== #
# st.set_page_config(page_title="🧠 AI Medical Diagnosis", layout="wide")

# # 🌈 Theme and custom UI
# st.markdown("""
#     <style>
#         .stApp {
#             background-color: #0a192f;
#             color: white;
#             padding: 20px;
#             border-radius: 15px;
#         }
#         .css-18e3th9, .css-1d391kg {
#             background-color: #112240 !important;
#             color: white !important;
#             border-radius: 10px;
#         }
#         .stTextInput>div>div>input, .stTextArea textarea {
#             color: white !important;
#             background-color: #233554 !important;
#         }
#         .block-container {
#             padding-top: 2rem;
#         }
#     </style>
# """, unsafe_allow_html=True)

# st.title("🩺 Advanced AI Medical Diagnosis (X-ray + Report + Fusion + Multiclass)")

# # =========================== 📥 Upload CSV and Show Preview ============================= #
# uploaded_csv = st.file_uploader("📄 Upload Radiology Report CSV", type="csv")
# tfidf, clf = None, None
# patient_ids = []

# if uploaded_csv:
#     df = pd.read_csv(uploaded_csv)
#     df['patient_id'] = ['PID' + str(random.randint(10000, 99999)) for _ in range(len(df))]
#     df['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#     df['age'] = np.random.randint(25, 80, size=len(df))
#     df['sex'] = np.random.choice(['Male', 'Female'], size=len(df))
#     df['symptoms'] = np.random.choice([
#         'Cough, Fever', 'Chest Pain, Breathlessness', 'Fatigue, Cough', 'Shortness of Breath', 'No major symptoms'
#     ], size=len(df))

#     st.subheader("📊 Sample Data")
#     st.dataframe(df.head(), use_container_width=True)

#     st.subheader("📈 Label Distribution")
#     fig, ax = plt.subplots(figsize=(4, 2.5))
#     df['label'].value_counts().plot(kind='bar', ax=ax, color=sns.color_palette("Set2"))
#     ax.set_xlabel("Label", color='white')
#     ax.set_ylabel("Count", color='white')
#     ax.tick_params(colors='white')
#     fig.patch.set_facecolor('#112240')
#     ax.set_facecolor('#112240')
#     st.pyplot(fig)

#     # ======================== 📊 EDA ============================= #
#     st.subheader("📌 Dataset Overview")
#     st.dataframe(df.describe(include='all'))

#     # =========================== 🧠 NLP Model ============================= #
#     st.subheader("🧠 Report Classification (TF-IDF + Logistic Regression)")
#     tfidf = TfidfVectorizer(max_features=300)
#     X = tfidf.fit_transform(df['report_text'])
#     y = df['label']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     clf = LogisticRegression(max_iter=1000)
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     probs = clf.predict_proba(X_test)
#     acc = accuracy_score(y_test, y_pred)
#     st.write("✅ Accuracy:", round(acc * 100, 2), "%")

#     st.subheader("📉 Confusion Matrix")
#     fig_cm, ax_cm = plt.subplots(figsize=(4, 2.5))
#     sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='coolwarm', ax=ax_cm, cbar=False)
#     ax_cm.set_xlabel("Predicted", color='white')
#     ax_cm.set_ylabel("Actual", color='white')
#     ax_cm.tick_params(colors='white')
#     fig_cm.patch.set_facecolor('#112240')
#     ax_cm.set_facecolor('#112240')
#     st.pyplot(fig_cm)

#     st.subheader("🧾 Classification Report")
#     st.dataframe(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

#     # =========================== 📊 Live Dashboard ============================= #
#     st.subheader("📈 Live Confidence Score Dashboard")
#     label_names = clf.classes_
#     class_probs = clf.predict_proba(X_test[:5])
#     for i in range(5):
#         st.markdown(f"**Patient {i+1}:**")
#         chart_data = pd.DataFrame({
#             'Label': label_names,
#             'Probability': class_probs[i]
#         })
#         st.bar_chart(chart_data.set_index('Label'))

# # =========================== 🖼️ CNN Model for X-ray Image ============================= #
# st.header("🩻 X-ray Diagnosis (CNN Binary)")

# @st.cache_resource
# def load_cnn_model():
#     model = resnet18(pretrained=True)
#     model.fc = nn.Linear(model.fc.in_features, 2)
#     return model

# cnn_model = load_cnn_model()
# cnn_model.eval()

# image_file = st.file_uploader("🖼️ Upload Chest X-ray Image", type=['png', 'jpg', 'jpeg'])
# pred_label = None
# if image_file:
#     image = Image.open(image_file).convert('RGB')
#     st.image(image, caption="Uploaded Chest X-ray", width=250)

#     transform = transforms.Compose([
#         transforms.Resize((124, 224)),
#         transforms.ToTensor(),
#     ])
#     input_tensor = transform(image).unsqueeze(0)
#     output = cnn_model(input_tensor)
#     pred_label = torch.argmax(output, 1).item()
#     st.success("✅ CNN Prediction: " + ("Normal" if pred_label == 0 else "⚠️ Abnormal"))

# # =========================== 🔀 Fusion Model ============================= #
# st.header("🔬 Multimodal Prediction (Fusion: X-ray + Report)")

# user_text = st.text_area("✍️ Enter a Radiology Report")
# if st.button("🔍 Predict from Report + X-ray"):
#     if user_text.strip() != "" and tfidf and clf:
#         text_pred = clf.predict(tfidf.transform([user_text]))[0]
#         prob = clf.predict_proba(tfidf.transform([user_text]))[0]
#         confidence = round(np.max(prob) * 100, 2)
#         fusion_result = f"🧠 NLP: {text_pred} ({confidence}%)"
#         if pred_label is not None:
#             fusion_result += f" | 🩻 CNN: {'Normal' if pred_label == 0 else 'Abnormal'}"
#         st.success("✅ Combined Prediction: " + fusion_result)

#         st.progress(int(confidence))
#     else:
#         st.warning("⚠️ Please upload X-ray + enter report")

# # =========================== 📤 Export Predictions ============================= #
# if uploaded_csv and st.button("📥 Export Predictions as CSV + PDF"):
#     df['predicted_label'] = clf.predict(tfidf.transform(df['report_text']))
#     df.to_csv("predicted_multimodal.csv", index=False)
#     st.success("✅ Exported as 'predicted_multimodal.csv'")

#     pdf = FPDF()
#     pdf.set_auto_page_break(auto=True, margin=15)
#     for i, row in df.iterrows():
#         pdf.add_page()
#         pdf.set_font("Arial", size=12)
#         pdf.cell(200, 10, txt=f"Patient ID: {row['patient_id']}", ln=True)
#         pdf.cell(200, 10, txt=f"Timestamp: {row['timestamp']}", ln=True)
#         pdf.cell(200, 10, txt=f"Age: {row['age']} | Sex: {row['sex']}", ln=True)
#         pdf.cell(200, 10, txt=f"Symptoms: {row['symptoms']}", ln=True)
#         pdf.cell(200, 10, txt=f"Report: {row['report_text'][:100]}...", ln=True)
#         pdf.cell(200, 10, txt=f"Predicted Diagnosis: {row['predicted_label']}", ln=True)
#     pdf.output("predicted_reports.pdf")
#     st.success("📄 PDF Exported as 'predicted_reports.pdf'")


In [6]:

# ################ 11th


# # ✅ All-in-One AI Medical Diagnosis (Multimodal Fusion + X-ray + Report + Multiclass + EDA + Export + DB Ready)

# import streamlit as st
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torchvision.transforms as transforms
# from torchvision.models import resnet18
# from PIL import Image
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# import random
# from sklearn.model_selection import train_test_split
# from fpdf import FPDF
# from datetime import datetime

# # ====================== 🌈 Streamlit UI Setup ====================== #
# st.set_page_config(page_title="🧠 AI Medical Diagnosis", layout="wide")
# st.markdown("""
#     <style>
#         .stApp {
#             background-color: #0a192f;
#             color: white;
#             padding: 20px;
#             border-radius: 15px;
#         }
#         .css-18e3th9, .css-1d391kg {
#             background-color: #112240 !important;
#             color: white !important;
#             border-radius: 10px;
#         }
#         .stTextInput>div>div>input, .stTextArea textarea {
#             color: white !important;
#             background-color: #233554 !important;
#         }
#         .block-container {
#             padding-top: 2rem;
#         }
#     </style>
# """, unsafe_allow_html=True)

# st.title("🩺 Advanced AI Medical Diagnosis (X-ray + Report + Fusion + Multiclass)")

# # =========================== 📥 Upload CSV and Show Preview ============================= #
# uploaded_csv = st.file_uploader("📄 Upload Radiology Report CSV", type="csv")
# tfidf, clf = None, None
# patient_ids = []

# if uploaded_csv:
#     df = pd.read_csv(uploaded_csv)
#     df['patient_id'] = ['PID' + str(random.randint(10000, 99999)) for _ in range(len(df))]
#     df['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#     df['age'] = np.random.randint(25, 80, size=len(df))
#     df['sex'] = np.random.choice(['Male', 'Female'], size=len(df))
#     df['symptoms'] = np.random.choice([
#         'Cough, Fever', 'Chest Pain, Breathlessness', 'Fatigue, Cough', 'Shortness of Breath', 'No major symptoms'
#     ], size=len(df))

#     st.subheader("📊 Sample Data")
#     st.dataframe(df.head(), use_container_width=True)

#     st.subheader("📈 Label Distribution")
#     fig, ax = plt.subplots(figsize=(4, 2.5))
#     df['label'].value_counts().plot(kind='bar', ax=ax, color=sns.color_palette("Set2"))
#     ax.set_xlabel("Label", color='white')
#     ax.set_ylabel("Count", color='white')
#     ax.tick_params(colors='white')
#     fig.patch.set_facecolor('#112240')
#     ax.set_facecolor('#112240')
#     st.pyplot(fig)

#     # ======================== 📊 EDA ============================= #
#     st.subheader("📌 Dataset Overview")
#     st.dataframe(df.describe(include='all'))

#     # =========================== 🧠 NLP Model ============================= #
#     st.subheader("🧠 Report Classification (TF-IDF + Logistic Regression)")
#     tfidf = TfidfVectorizer(max_features=300)
#     X = tfidf.fit_transform(df['report_text'])
#     y = df['label']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     clf = LogisticRegression(max_iter=1000)
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     probs = clf.predict_proba(X_test)
#     acc = accuracy_score(y_test, y_pred)
#     st.write("✅ Accuracy:", round(acc * 100, 2), "%")

#     st.subheader("📉 Confusion Matrix")
#     fig_cm, ax_cm = plt.subplots(figsize=(4, 2.5))
#     sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='coolwarm', ax=ax_cm, cbar=False)
#     ax_cm.set_xlabel("Predicted", color='white')
#     ax_cm.set_ylabel("Actual", color='white')
#     ax_cm.tick_params(colors='white')
#     fig_cm.patch.set_facecolor('#112240')
#     ax_cm.set_facecolor('#112240')
#     st.pyplot(fig_cm)

#     st.subheader("🧾 Classification Report")
#     st.dataframe(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

#     # =========================== 📊 Live Dashboard ============================= #
#     st.subheader("📈 Live Confidence Score Dashboard")
#     label_names = clf.classes_
#     class_probs = clf.predict_proba(X_test[:5])
#     for i in range(5):
#         st.markdown(f"**Patient {i+1}:**")
#         chart_data = pd.DataFrame({
#             'Label': label_names,
#             'Probability': class_probs[i]
#         })
#         st.bar_chart(chart_data.set_index('Label'))

# # =========================== 🖼️ CNN Model for X-ray Image ============================= #
# st.header("🩻 X-ray Diagnosis (CNN Binary)")

# @st.cache_resource
# def load_cnn_model():
#     model = resnet18(pretrained=True)
#     model.fc = nn.Linear(model.fc.in_features, 2)
#     return model

# cnn_model = load_cnn_model()
# cnn_model.eval()

# image_file = st.file_uploader("🖼️ Upload Chest X-ray Image", type=['png', 'jpg', 'jpeg'])
# pred_label = None
# if image_file:
#     image = Image.open(image_file).convert('RGB')
#     st.image(image, caption="Uploaded Chest X-ray", width=250)

#     transform = transforms.Compose([
#         transforms.Resize((124, 224)),
#         transforms.ToTensor(),
#     ])
#     input_tensor = transform(image).unsqueeze(0)
#     output = cnn_model(input_tensor)
#     pred_label = torch.argmax(output, 1).item()
#     st.success("✅ CNN Prediction: " + ("Normal" if pred_label == 0 else "⚠️ Abnormal"))

# # =========================== 🔀 Fusion Model ============================= #
# st.header("🔬 Multimodal Prediction (Fusion: X-ray + Report)")

# user_text = st.text_area("✍️ Enter a Radiology Report")
# if st.button("🔍 Predict from Report + X-ray"):
#     if user_text.strip() != "" and tfidf and clf:
#         text_pred = clf.predict(tfidf.transform([user_text]))[0]
#         prob = clf.predict_proba(tfidf.transform([user_text]))[0]
#         confidence = round(np.max(prob) * 100, 2)
#         fusion_result = f"🧠 NLP: {text_pred} ({confidence}%)"
#         if pred_label is not None:
#             fusion_result += f" | 🩻 CNN: {'Normal' if pred_label == 0 else 'Abnormal'}"
#         st.success("✅ Combined Prediction: " + fusion_result)

#         st.progress(int(confidence))
#     else:
#         st.warning("⚠️ Please upload X-ray + enter report")

# # =========================== 📤 Export Predictions ============================= #
# if uploaded_csv and st.button("📥 Export Predictions as CSV + PDF"):
#     df['predicted_label'] = clf.predict(tfidf.transform(df['report_text']))
#     df.to_csv("predicted_multimodal.csv", index=False)
#     st.success("✅ Exported as 'predicted_multimodal.csv'")

#     pdf = FPDF()
#     pdf.set_auto_page_break(auto=True, margin=15)
#     for i, row in df.iterrows():
#         pdf.add_page()
#         pdf.set_font("Arial", size=12)
#         pdf.cell(200, 10, txt=f"Patient ID: {row['patient_id']}", ln=True)
#         pdf.cell(200, 10, txt=f"Timestamp: {row['timestamp']}", ln=True)
#         pdf.cell(200, 10, txt=f"Age: {row['age']} | Sex: {row['sex']}", ln=True)
#         pdf.cell(200, 10, txt=f"Symptoms: {row['symptoms']}", ln=True)
#         pdf.cell(200, 10, txt=f"Report: {row['report_text'][:100]}...", ln=True)
#         pdf.cell(200, 10, txt=f"Predicted Diagnosis: {row['predicted_label']}", ln=True)
#     pdf.output("predicted_reports.pdf")
#     st.success("📄 PDF Exported as 'predicted_reports.pdf'")



In [7]:

# ################## 10th

# # ✅ All-in-One AI Medical Diagnosis (Multimodal Fusion + X-ray + Report + Multiclass + EDA + Export + DB Ready)

# import streamlit as st
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torchvision.transforms as transforms
# from torchvision.models import resnet18
# from PIL import Image
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# import random
# from sklearn.model_selection import train_test_split
# from fpdf import FPDF
# from datetime import datetime

# # ====================== 🌈 Streamlit UI Setup ====================== #
# st.set_page_config(page_title="🧠 AI Medical Diagnosis", layout="wide")
# st.markdown("""
#     <style>
#         .stApp {
#             background-color: #0a192f;
#             color: white;
#             padding: 20px;
#             border-radius: 15px;
#         }
#         .css-18e3th9, .css-1d391kg {
#             background-color: #112240 !important;
#             color: white !important;
#             border-radius: 10px;
#         }
#         .stTextInput>div>div>input, .stTextArea textarea {
#             color: white !important;
#             background-color: #233554 !important;
#         }
#         .block-container {
#             padding-top: 2rem;
#         }
#     </style>
# """, unsafe_allow_html=True)

# st.title("🩺 Advanced AI Medical Diagnosis (X-ray + Report + Fusion + Multiclass)")

# # =========================== 📥 Upload CSV and Show Preview ============================= #
# uploaded_csv = st.file_uploader("📄 Upload Radiology Report CSV", type="csv")
# tfidf, clf = None, None
# patient_ids = []

# if uploaded_csv:
#     df = pd.read_csv(uploaded_csv)
#     df['patient_id'] = ['PID' + str(random.randint(10000, 99999)) for _ in range(len(df))]
#     df['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#     df['age'] = np.random.randint(25, 80, size=len(df))
#     df['sex'] = np.random.choice(['Male', 'Female'], size=len(df))
#     df['symptoms'] = np.random.choice([
#         'Cough, Fever', 'Chest Pain, Breathlessness', 'Fatigue, Cough', 'Shortness of Breath', 'No major symptoms'
#     ], size=len(df))

#     st.subheader("📊 Sample Data")
#     st.dataframe(df.head(), use_container_width=True)

#     st.subheader("📈 Label Distribution")
#     fig, ax = plt.subplots(figsize=(4, 2.5))
#     df['label'].value_counts().plot(kind='bar', ax=ax, color=sns.color_palette("Set2"))
#     ax.set_xlabel("Label", color='white')
#     ax.set_ylabel("Count", color='white')
#     ax.tick_params(colors='white')
#     fig.patch.set_facecolor('#112240')
#     ax.set_facecolor('#112240')
#     st.pyplot(fig)

#     # ======================== 📊 EDA ============================= #
#     st.subheader("📌 Dataset Overview")
#     st.dataframe(df.describe(include='all'))

#     # =========================== 🧠 NLP Model ============================= #
#     st.subheader("🧠 Report Classification (TF-IDF + Logistic Regression)")
#     tfidf = TfidfVectorizer(max_features=300)
#     X = tfidf.fit_transform(df['report_text'])
#     y = df['label']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     clf = LogisticRegression(max_iter=1000)
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     probs = clf.predict_proba(X_test)
#     acc = accuracy_score(y_test, y_pred)
#     st.write("✅ Accuracy:", round(acc * 100, 2), "%")

#     st.subheader("📉 Confusion Matrix")
#     fig_cm, ax_cm = plt.subplots(figsize=(4, 2.5))
#     sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='coolwarm', ax=ax_cm, cbar=False)
#     ax_cm.set_xlabel("Predicted", color='white')
#     ax_cm.set_ylabel("Actual", color='white')
#     ax_cm.tick_params(colors='white')
#     fig_cm.patch.set_facecolor('#112240')
#     ax_cm.set_facecolor('#112240')
#     st.pyplot(fig_cm)

#     st.subheader("🧾 Classification Report")
#     st.dataframe(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

# # =========================== 🖼️ CNN Model for X-ray Image ============================= #
# st.header("🩻 X-ray Diagnosis (CNN Binary)")

# @st.cache_resource
# def load_cnn_model():
#     model = resnet18(pretrained=True)
#     model.fc = nn.Linear(model.fc.in_features, 2)
#     return model

# cnn_model = load_cnn_model()
# cnn_model.eval()

# image_file = st.file_uploader("🖼️ Upload Chest X-ray Image", type=['png', 'jpg', 'jpeg'])
# pred_label = None
# if image_file:
#     image = Image.open(image_file).convert('RGB')
#     st.image(image, caption="Uploaded Chest X-ray", width=250)

#     transform = transforms.Compose([
#         transforms.Resize((124, 224)),
#         transforms.ToTensor(),
#     ])
#     input_tensor = transform(image).unsqueeze(0)
#     output = cnn_model(input_tensor)
#     pred_label = torch.argmax(output, 1).item()
#     st.success("✅ CNN Prediction: " + ("Normal" if pred_label == 0 else "⚠️ Abnormal"))

# # =========================== 🔀 Fusion Model ============================= #
# st.header("🔬 Multimodal Prediction (Fusion: X-ray + Report)")

# user_text = st.text_area("✍️ Enter a Radiology Report")
# if st.button("🔍 Predict from Report + X-ray"):
#     if user_text.strip() != "" and tfidf and clf:
#         text_pred = clf.predict(tfidf.transform([user_text]))[0]
#         prob = clf.predict_proba(tfidf.transform([user_text]))[0]
#         confidence = round(np.max(prob) * 100, 2)
#         fusion_result = f"🧠 NLP: {text_pred} ({confidence}%)"
#         if pred_label is not None:
#             fusion_result += f" | 🩻 CNN: {'Normal' if pred_label == 0 else 'Abnormal'}"
#         st.success("✅ Combined Prediction: " + fusion_result)

#         st.progress(int(confidence))
#     else:
#         st.warning("⚠️ Please upload X-ray + enter report")

# # =========================== 📤 Export Predictions ============================= #
# if uploaded_csv and st.button("📥 Export Predictions as CSV + PDF"):
#     df['predicted_label'] = clf.predict(tfidf.transform(df['report_text']))
#     df.to_csv("predicted_multimodal.csv", index=False)
#     st.success("✅ Exported as 'predicted_multimodal.csv'")

#     pdf = FPDF()
#     pdf.set_auto_page_break(auto=True, margin=15)
#     for i, row in df.iterrows():
#         pdf.add_page()
#         pdf.set_font("Arial", size=12)
#         pdf.cell(200, 10, txt=f"Patient ID: {row['patient_id']}", ln=True)
#         pdf.cell(200, 10, txt=f"Timestamp: {row['timestamp']}", ln=True)
#         pdf.cell(200, 10, txt=f"Age: {row['age']} | Sex: {row['sex']}", ln=True)
#         pdf.cell(200, 10, txt=f"Symptoms: {row['symptoms']}", ln=True)
#         pdf.cell(200, 10, txt=f"Report: {row['report_text'][:100]}...", ln=True)
#         pdf.cell(200, 10, txt=f"Predicted Diagnosis: {row['predicted_label']}", ln=True)
#     pdf.output("predicted_reports.pdf")
#     st.success("📄 PDF Exported as 'predicted_reports.pdf'")



In [8]:

# ############################# 9th

# # ✅ All-in-One AI Medical Diagnosis (Multimodal Fusion + X-ray + Report + Multiclass + EDA + Export + DB Ready)

# import streamlit as st
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torchvision.transforms as transforms
# from torchvision.models import resnet18
# from PIL import Image
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# import random
# from sklearn.model_selection import train_test_split
# from fpdf import FPDF
# from datetime import datetime

# # ====================== 🌈 Streamlit UI Setup ====================== #
# st.set_page_config(page_title="🧠 AI Medical Diagnosis", layout="wide")
# st.markdown("""
#     <style>
#         .stApp {
#             background-color: #0a192f;
#             color: white;
#             padding: 20px;
#             border-radius: 15px;
#         }
#         .css-18e3th9, .css-1d391kg {
#             background-color: #112240 !important;
#             color: white !important;
#             border-radius: 10px;
#         }
#         .stTextInput>div>div>input, .stTextArea textarea {
#             color: white !important;
#             background-color: #233554 !important;
#         }
#         .block-container {
#             padding-top: 2rem;
#         }
#     </style>
# """, unsafe_allow_html=True)

# st.title("🩺 Advanced AI Medical Diagnosis (X-ray + Report + Fusion + Multiclass)")

# # =========================== 📥 Upload CSV and Show Preview ============================= #
# uploaded_csv = st.file_uploader("📄 Upload Radiology Report CSV", type="csv")
# tfidf, clf = None, None
# patient_ids = []

# if uploaded_csv:
#     df = pd.read_csv(uploaded_csv)
#     df['patient_id'] = ['PID' + str(random.randint(10000, 99999)) for _ in range(len(df))]
#     df['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#     st.subheader("📊 Sample Data")
#     st.dataframe(df.head(), use_container_width=True)

#     st.subheader("📈 Label Distribution")
#     fig, ax = plt.subplots(figsize=(4, 2.5))
#     df['label'].value_counts().plot(kind='bar', ax=ax, color=sns.color_palette("Set2"))
#     ax.set_xlabel("Label", color='white')
#     ax.set_ylabel("Count", color='white')
#     ax.tick_params(colors='white')
#     fig.patch.set_facecolor('#112240')
#     ax.set_facecolor('#112240')
#     st.pyplot(fig)

#     # ======================== 📊 EDA ============================= #
#     st.subheader("📌 Dataset Overview")
#     st.dataframe(df.describe(include='all'))

#     # =========================== 🧠 NLP Model ============================= #
#     st.subheader("🧠 Report Classification (TF-IDF + Logistic Regression)")
#     tfidf = TfidfVectorizer(max_features=300)
#     X = tfidf.fit_transform(df['report_text'])
#     y = df['label']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     clf = LogisticRegression(max_iter=1000)
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     probs = clf.predict_proba(X_test)
#     acc = accuracy_score(y_test, y_pred)
#     st.write("✅ Accuracy:", round(acc * 100, 2), "%")

#     st.subheader("📉 Confusion Matrix")
#     fig_cm, ax_cm = plt.subplots(figsize=(4, 2.5))
#     sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='coolwarm', ax=ax_cm, cbar=False)
#     ax_cm.set_xlabel("Predicted", color='white')
#     ax_cm.set_ylabel("Actual", color='white')
#     ax_cm.tick_params(colors='white')
#     fig_cm.patch.set_facecolor('#112240')
#     ax_cm.set_facecolor('#112240')
#     st.pyplot(fig_cm)

#     st.subheader("🧾 Classification Report")
#     st.dataframe(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

# # =========================== 🖼️ CNN Model for X-ray Image ============================= #
# st.header("🩻 X-ray Diagnosis (CNN Binary)")

# @st.cache_resource
# def load_cnn_model():
#     model = resnet18(pretrained=True)
#     model.fc = nn.Linear(model.fc.in_features, 2)
#     return model

# cnn_model = load_cnn_model()
# cnn_model.eval()

# image_file = st.file_uploader("🖼️ Upload Chest X-ray Image", type=['png', 'jpg', 'jpeg'])
# pred_label = None
# if image_file:
#     image = Image.open(image_file).convert('RGB')
#     st.image(image, caption="Uploaded Chest X-ray", width=250)

#     transform = transforms.Compose([
#         transforms.Resize((124, 224)),
#         transforms.ToTensor(),
#     ])
#     input_tensor = transform(image).unsqueeze(0)
#     output = cnn_model(input_tensor)
#     pred_label = torch.argmax(output, 1).item()
#     st.success("✅ CNN Prediction: " + ("Normal" if pred_label == 0 else "⚠️ Abnormal"))

# # =========================== 🔀 Fusion Model ============================= #
# st.header("🔬 Multimodal Prediction (Fusion: X-ray + Report)")

# user_text = st.text_area("✍️ Enter a Radiology Report")
# if st.button("🔍 Predict from Report + X-ray"):
#     if user_text.strip() != "" and tfidf and clf:
#         text_pred = clf.predict(tfidf.transform([user_text]))[0]
#         prob = clf.predict_proba(tfidf.transform([user_text]))[0]
#         confidence = round(np.max(prob) * 100, 2)
#         fusion_result = f"🧠 NLP: {text_pred} ({confidence}%)"
#         if pred_label is not None:
#             fusion_result += f" | 🩻 CNN: {'Normal' if pred_label == 0 else 'Abnormal'}"
#         st.success("✅ Combined Prediction: " + fusion_result)

#         st.progress(int(confidence))
#     else:
#         st.warning("⚠️ Please upload X-ray + enter report")

# # =========================== 📤 Export Predictions ============================= #
# if uploaded_csv and st.button("📥 Export Predictions as CSV + PDF"):
#     df['predicted_label'] = clf.predict(tfidf.transform(df['report_text']))
#     df.to_csv("predicted_multimodal.csv", index=False)
#     st.success("✅ Exported as 'predicted_multimodal.csv'")

#     pdf = FPDF()
#     pdf.set_auto_page_break(auto=True, margin=15)
#     for i, row in df.iterrows():
#         pdf.add_page()
#         pdf.set_font("Arial", size=12)
#         pdf.cell(200, 10, txt=f"Patient ID: {row['patient_id']}", ln=True)
#         pdf.cell(200, 10, txt=f"Timestamp: {row['timestamp']}", ln=True)
#         pdf.cell(200, 10, txt=f"Report: {row['report_text'][:100]}...", ln=True)
#         pdf.cell(200, 10, txt=f"Predicted Diagnosis: {row['predicted_label']}", ln=True)
#     pdf.output("predicted_reports.pdf")
#     st.success("📄 PDF Exported as 'predicted_reports.pdf'")


In [9]:

# ################# 8th

# # ✅ All-in-One AI Medical Diagnosis (Multimodal Fusion + X-ray + Report + Multiclass + EDA + Export)

# import streamlit as st
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torchvision.transforms as transforms
# from torchvision.models import resnet18
# from PIL import Image
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# import random
# from sklearn.model_selection import train_test_split
# from fpdf import FPDF

# # ====================== 🌈 Streamlit UI Setup ====================== #
# st.set_page_config(page_title="🧠 AI Medical Diagnosis", layout="wide")
# st.markdown("""
#     <style>
#         .stApp {
#             background-color: #0d1117;
#             color: white;
#             padding: 20px;
#             border-radius: 15px;
#         }
#         .css-18e3th9, .css-1d391kg {
#             background-color: #161b22 !important;
#             color: white !important;
#             border-radius: 10px;
#         }
#         .stTextInput>div>div>input, .stTextArea textarea {
#             color: white !important;
#             background-color: #21262d !important;
#         }
#         .block-container {
#             padding-top: 2rem;
#         }
#     </style>
# """, unsafe_allow_html=True)

# st.title("🩺 Advanced AI Medical Diagnosis (X-ray + Report + Fusion + Multiclass)")

# # =========================== 📥 Upload CSV and Show Preview ============================= #
# uploaded_csv = st.file_uploader("📄 Upload Radiology Report CSV", type="csv")
# tfidf, clf = None, None
# patient_ids = []

# if uploaded_csv:
#     df = pd.read_csv(uploaded_csv)
#     df['patient_id'] = ['PID' + str(random.randint(10000, 99999)) for _ in range(len(df))]
#     st.subheader("📊 Sample Data")
#     st.dataframe(df.head(), use_container_width=True)

#     st.subheader("📈 Label Distribution")
#     fig, ax = plt.subplots(figsize=(3, 2))
#     df['label'].value_counts().plot(kind='bar', ax=ax, color=sns.color_palette("Set2"))
#     ax.set_xlabel("Label", color='white')
#     ax.set_ylabel("Count", color='white')
#     ax.tick_params(colors='white')
#     fig.patch.set_facecolor('#161b22')
#     ax.set_facecolor('#161b22')
#     st.pyplot(fig)

#     # ======================== 📊 EDA ============================= #
#     st.subheader("📌 Dataset Overview")
#     st.dataframe(df.describe(include='all'))

#     # =========================== 🧠 NLP Model ============================= #
#     st.subheader("🧠 Report Classification (TF-IDF + Logistic Regression)")
#     tfidf = TfidfVectorizer(max_features=300)
#     X = tfidf.fit_transform(df['report_text'])
#     y = df['label']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     clf = LogisticRegression(max_iter=1000)
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     probs = clf.predict_proba(X_test)
#     acc = accuracy_score(y_test, y_pred)
#     st.write("✅ Accuracy:", round(acc * 100, 2), "%")

#     st.subheader("📉 Confusion Matrix")
#     fig_cm, ax_cm = plt.subplots(figsize=(3, 2))
#     sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='coolwarm', ax=ax_cm, cbar=False)
#     ax_cm.set_xlabel("Predicted", color='white')
#     ax_cm.set_ylabel("Actual", color='white')
#     ax_cm.tick_params(colors='white')
#     fig_cm.patch.set_facecolor('#161b22')
#     ax_cm.set_facecolor('#161b22')
#     st.pyplot(fig_cm)

#     st.subheader("🧾 Classification Report")
#     st.dataframe(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

# # =========================== 🖼️ CNN Model for X-ray Image ============================= #
# st.header("🩻 X-ray Diagnosis (CNN Binary)")

# @st.cache_resource
# def load_cnn_model():
#     model = resnet18(pretrained=True)
#     model.fc = nn.Linear(model.fc.in_features, 2)
#     return model

# cnn_model = load_cnn_model()
# cnn_model.eval()

# image_file = st.file_uploader("🖼️ Upload Chest X-ray Image", type=['png', 'jpg', 'jpeg'])
# pred_label = None
# if image_file:
#     image = Image.open(image_file).convert('RGB')
#     st.image(image, caption="Uploaded Chest X-ray", width=250)

#     transform = transforms.Compose([
#         transforms.Resize((124, 224)),
#         transforms.ToTensor(),
#     ])
#     input_tensor = transform(image).unsqueeze(0)
#     output = cnn_model(input_tensor)
#     pred_label = torch.argmax(output, 1).item()
#     st.success("✅ CNN Prediction: " + ("Normal" if pred_label == 0 else "⚠️ Abnormal"))

# # =========================== 🔀 Fusion Model ============================= #
# st.header("🔬 Multimodal Prediction (Fusion: X-ray + Report)")

# user_text = st.text_area("✍️ Enter a Radiology Report")
# if st.button("🔍 Predict from Report + X-ray"):
#     if user_text.strip() != "" and tfidf and clf:
#         text_pred = clf.predict(tfidf.transform([user_text]))[0]
#         prob = clf.predict_proba(tfidf.transform([user_text]))[0]
#         confidence = round(np.max(prob) * 100, 2)
#         fusion_result = f"🧠 NLP: {text_pred} ({confidence}%)"
#         if pred_label is not None:
#             fusion_result += f" | 🩻 CNN: {'Normal' if pred_label == 0 else 'Abnormal'}"
#         st.success("✅ Combined Prediction: " + fusion_result)

#         st.progress(int(confidence))
#     else:
#         st.warning("⚠️ Please upload X-ray + enter report")

# # =========================== 📤 Export Predictions ============================= #
# if uploaded_csv and st.button("📥 Export Predictions as CSV + PDF"):
#     df['predicted_label'] = clf.predict(tfidf.transform(df['report_text']))
#     df.to_csv("predicted_multimodal.csv", index=False)
#     st.success("✅ Exported as 'predicted_multimodal.csv'")

#     pdf = FPDF()
#     pdf.set_auto_page_break(auto=True, margin=15)
#     for i, row in df.iterrows():
#         pdf.add_page()
#         pdf.set_font("Arial", size=12)
#         pdf.cell(200, 10, txt=f"Patient ID: {row['patient_id']}", ln=True)
#         pdf.cell(200, 10, txt=f"Report: {row['report_text'][:100]}...", ln=True)
#         pdf.cell(200, 10, txt=f"Predicted Diagnosis: {row['predicted_label']}", ln=True)
#     pdf.output("predicted_reports.pdf")
#     st.success("📄 PDF Exported as 'predicted_reports.pdf'")




In [10]:
# #########################  7th

# # ✅ All-in-One AI Medical Diagnosis (Multimodal Fusion + X-ray + Report + Multiclass + EDA)

# import streamlit as st
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torchvision.transforms as transforms
# from torchvision.models import resnet18
# from PIL import Image
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# from sklearn.model_selection import train_test_split

# # ====================== 🌈 Streamlit UI Setup ====================== #
# st.set_page_config(page_title="🧠 AI Medical Diagnosis", layout="wide")
# st.markdown("""
#     <style>
#         .stApp {
#             background-color: #0d1117;
#             color: white;
#             padding: 20px;
#             border-radius: 15px;
#         }
#         .css-18e3th9, .css-1d391kg {
#             background-color: #161b22 !important;
#             color: white !important;
#             border-radius: 10px;
#         }
#         .stTextInput>div>div>input, .stTextArea textarea {
#             color: white !important;
#             background-color: #21262d !important;
#         }
#         .block-container {
#             padding-top: 2rem;
#         }
#     </style>
# """, unsafe_allow_html=True)

# st.title("🩺 Advanced AI Medical Diagnosis (X-ray + Report + Fusion + Multiclass)")

# # =========================== 📥 Upload CSV and Show Preview ============================= #
# uploaded_csv = st.file_uploader("📄 Upload Radiology Report CSV", type="csv")
# tfidf, clf = None, None

# if uploaded_csv:
#     df = pd.read_csv(uploaded_csv)
#     st.subheader("📊 Sample Data")
#     st.dataframe(df.head(), use_container_width=True)

#     st.subheader("📈 Label Distribution")
#     fig, ax = plt.subplots(figsize=(3, 2))
#     df['label'].value_counts().plot(kind='bar', ax=ax, color=sns.color_palette("Set2"))
#     ax.set_xlabel("Label", color='white')
#     ax.set_ylabel("Count", color='white')
#     ax.tick_params(colors='white')
#     fig.patch.set_facecolor('#161b22')
#     ax.set_facecolor('#161b22')
#     st.pyplot(fig)

#     # ======================== 📊 EDA ============================= #
#     st.subheader("📌 Dataset Overview")
#     st.dataframe(df.describe(include='all'))

#     # =========================== 🧠 NLP Model ============================= #
#     st.subheader("🧠 Report Classification (TF-IDF + Logistic Regression)")
#     tfidf = TfidfVectorizer(max_features=300)
#     X = tfidf.fit_transform(df['report_text'])
#     y = df['label']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     clf = LogisticRegression(max_iter=1000)
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     acc = accuracy_score(y_test, y_pred)
#     st.write("✅ Accuracy:", round(acc * 100, 2), "%")

#     st.subheader("📉 Confusion Matrix")
#     fig_cm, ax_cm = plt.subplots(figsize=(3, 2))
#     sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='coolwarm', ax=ax_cm, cbar=False)
#     ax_cm.set_xlabel("Predicted", color='white')
#     ax_cm.set_ylabel("Actual", color='white')
#     ax_cm.tick_params(colors='white')
#     fig_cm.patch.set_facecolor('#161b22')
#     ax_cm.set_facecolor('#161b22')
#     st.pyplot(fig_cm)

#     st.subheader("🧾 Classification Report")
#     st.dataframe(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

# # =========================== 🖼️ CNN Model for X-ray Image ============================= #
# st.header("🩻 X-ray Diagnosis (CNN Binary)")

# @st.cache_resource
# def load_cnn_model():
#     model = resnet18(pretrained=True)
#     model.fc = nn.Linear(model.fc.in_features, 2)
#     return model

# cnn_model = load_cnn_model()
# cnn_model.eval()

# image_file = st.file_uploader("🖼️ Upload Chest X-ray Image", type=['png', 'jpg', 'jpeg'])
# pred_label = None
# if image_file:
#     image = Image.open(image_file).convert('RGB')
#     st.image(image, caption="Uploaded Chest X-ray", width=250)

#     transform = transforms.Compose([
#         transforms.Resize((124, 224)),
#         transforms.ToTensor(),
#     ])
#     input_tensor = transform(image).unsqueeze(0)
#     output = cnn_model(input_tensor)
#     pred_label = torch.argmax(output, 1).item()
#     st.success("✅ CNN Prediction: " + ("Normal" if pred_label == 0 else "⚠️ Abnormal"))

# # =========================== 🔀 Fusion Model ============================= #
# st.header("🔬 Multimodal Prediction (Fusion: X-ray + Report)")

# user_text = st.text_area("✍️ Enter a Radiology Report")
# if st.button("🔍 Predict from Report + X-ray"):
#     if user_text.strip() != "" and tfidf and clf:
#         text_pred = clf.predict(tfidf.transform([user_text]))[0]
#         fusion_result = f"🧠 NLP: {text_pred}"
#         if pred_label is not None:
#             fusion_result += f" | 🩻 CNN: {'Normal' if pred_label == 0 else 'Abnormal'}"
#         st.success("✅ Combined Prediction: " + fusion_result)
#     else:
#         st.warning("⚠️ Please upload X-ray + enter report")

# # =========================== 📤 Export Predictions ============================= #
# if uploaded_csv and st.button("📥 Export Predictions as CSV"):
#     df['predicted_label'] = clf.predict(tfidf.transform(df['report_text']))
#     df.to_csv("predicted_multimodal.csv", index=False)
#     st.success("✅ Exported as 'predicted_multimodal.csv'")



In [11]:
############## 6th
# # ✅ All-in-One AI Medical Diagnosis (X-ray + Report Analysis + EDA + Metrics + Top UI)

# import streamlit as st
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torchvision.transforms as transforms
# from torchvision.models import resnet18
# from PIL import Image
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os

# # ====================== 🌈 Streamlit UI Setup ====================== #
# st.set_page_config(page_title="🧠 AI Medical Diagnosis", layout="wide")
# st.markdown("""
#     <style>
#         .stApp {
#             background-color: #212F3D;
#             color: white;
#             padding: 20px;
#             border-radius: 15px;
#         }
#         .css-18e3th9, .css-1d391kg {
#             background-color: #2C3E50 !important;
#             color: white !important;
#             border-radius: 10px;
#         }
#         .stTextInput>div>div>input {
#             color: white !important;
#             background-color: #566573 !important;
#         }
#     </style>
# """, unsafe_allow_html=True)

# st.title("🩺 AI-Based Medical Diagnosis (Chest X-ray + Report NLP)")

# # =========================== 📥 Upload CSV and Show Preview ============================= #
# uploaded_csv = st.file_uploader("📄 Upload Radiology Report CSV", type="csv")
# if uploaded_csv:
#     df = pd.read_csv(uploaded_csv)
#     st.subheader("📊 Sample Data")
#     st.dataframe(df.head(), use_container_width=True)

#     st.subheader("📈 Label Distribution")
#     fig, ax = plt.subplots(figsize=(2, 1))
#     df['label'].value_counts().plot(kind='bar', ax=ax, color=['#4CAF50', '#FF5733'])
#     ax.set_xlabel("Label (0=Normal, 1=Abnormal)", color='white')
#     ax.set_ylabel("Count", color='white')
#     ax.tick_params(colors='white')
#     fig.patch.set_facecolor('#2C3E50')
#     ax.set_facecolor('#2C3E50')
#     st.pyplot(fig)

#     # ======================== 📊 EDA and Column Profiling ============================= #
#     st.subheader("📌 Column-wise Info")
#     st.dataframe(df.describe(include='all'))

#     # =========================== 🧠 NLP - Report Text Classification ============================= #
#     st.subheader("🧠 Radiology Report Classifier (NLP)")
#     tfidf = TfidfVectorizer(max_features=300)
#     X = tfidf.fit_transform(df['report_text'])
#     y = df['label']

#     from sklearn.model_selection import train_test_split
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     clf = LogisticRegression()
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     acc = accuracy_score(y_test, y_pred)
#     st.write("✅ Accuracy:", round(acc * 100, 2), "%")

#     # Confusion Matrix (Smaller)
#     cm = confusion_matrix(y_test, y_pred)
#     st.subheader("📉 Confusion Matrix")
#     fig_cm, ax_cm = plt.subplots(figsize=(2, 1))
#     sns.heatmap(cm, annot=True, fmt='d', cmap='cool', ax=ax_cm, cbar=False)
#     ax_cm.set_xlabel("Predicted", color='white')
#     ax_cm.set_ylabel("Actual", color='white')
#     ax_cm.tick_params(colors='white')
#     fig_cm.patch.set_facecolor('#2C3E50')
#     ax_cm.set_facecolor('#2C3E50')
#     st.pyplot(fig_cm)

#     # Classification Report
#     st.subheader("🧾 Classification Report")
#     st.dataframe(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

# # =========================== 🖼️ CNN Model for X-ray Image ============================= #
# st.header("🩻 Chest X-ray Diagnosis (CNN)")

# @st.cache_resource
# def load_cnn_model():
#     model = resnet18(pretrained=True)
#     model.fc = nn.Linear(model.fc.in_features, 2)
#     return model

# model = load_cnn_model()
# model.eval()

# image_file = st.file_uploader("🖼️ Upload Chest X-ray Image", type=['png', 'jpg', 'jpeg'])
# if image_file:
#     image = Image.open(image_file).convert('RGB')
#     st.image(image, caption="Uploaded Chest X-ray", width=250)

#     transform = transforms.Compose([
#         transforms.Resize((124, 224)),
#         transforms.ToTensor(),
#     ])
#     input_tensor = transform(image).unsqueeze(0)
#     output = model(input_tensor)
#     pred_label = torch.argmax(output, 1).item()
#     st.success("✅ Prediction: " + ("Normal" if pred_label == 0 else "⚠️ Abnormal"))

# # =========================== 🧪 Try with Custom Report ============================= #
# st.header("📝 Try with Custom Radiology Report")

# user_text = st.text_area("Enter a Radiology Report")
# if st.button("🔍 Analyze Report"):
#     if user_text.strip() != "":
#         input_vector = tfidf.transform([user_text])
#         pred = clf.predict(input_vector)[0]
#         st.success("✅ Report Prediction: " + ("Normal" if pred == 0 else "⚠️ Abnormal"))
#     else:
#         st.warning("⚠️ Please enter a report.")

# # =========================== 📤 Export Predictions ============================= #
# if uploaded_csv and st.button("📥 Export CSV with Predictions"):
#     df['predicted_label'] = clf.predict(tfidf.transform(df['report_text']))
#     df.to_csv("predicted_reports.csv", index=False)
#     st.success("✅ Exported as 'predicted_reports.csv'")


In [12]:
# ############### 5th 

# # ✅ All-in-One AI Medical Diagnosis (X-ray + Report Analysis + EDA + Metrics)

# import streamlit as st
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torchvision.transforms as transforms
# from torchvision.models import resnet18
# from PIL import Image
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os

# st.set_page_config(page_title="🧠 AI Medical Diagnosis", layout="wide")
# st.title("🩺 AI-Based Medical Diagnosis (Chest X-ray + Report NLP)")

# # =========================== 📥 Upload CSV and Show Preview ============================= #
# uploaded_csv = st.file_uploader("📄 Upload Radiology Report CSV", type="csv")
# if uploaded_csv:
#     df = pd.read_csv(uploaded_csv)
#     st.subheader("📊 Sample Data")
#     st.dataframe(df.head(), use_container_width=True)

#     st.subheader("📈 Label Distribution")
#     fig, ax = plt.subplots(figsize=(2.5, 1.5))
#     df['label'].value_counts().plot(kind='bar', ax=ax)
#     ax.set_xlabel("Label (0=Normal, 1=Abnormal)")
#     ax.set_ylabel("Count")
#     st.pyplot(fig)

#     # ======================== 📊 EDA and Column Profiling ============================= #
#     st.subheader("📌 Column-wise Info")
#     st.dataframe(df.describe(include='all'))

#     # =========================== 🧠 NLP - Report Text Classification ============================= #
#     st.subheader("🧠 Radiology Report Classifier (NLP)")
#     tfidf = TfidfVectorizer(max_features=300)
#     X = tfidf.fit_transform(df['report_text'])
#     y = df['label']

#     from sklearn.model_selection import train_test_split
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     clf = LogisticRegression()
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     acc = accuracy_score(y_test, y_pred)
#     st.write("✅ Accuracy:", round(acc * 100, 2), "%")

#     # Confusion Matrix (Smaller)
#     cm = confusion_matrix(y_test, y_pred)
#     st.subheader("📉 Confusion Matrix")
#     fig_cm, ax_cm = plt.subplots(figsize=(2.5, 1.5))
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax_cm, cbar=False)
#     ax_cm.set_xlabel("Predicted")
#     ax_cm.set_ylabel("Actual")
#     st.pyplot(fig_cm)

#     # Classification Report
#     st.subheader("🧾 Classification Report")
#     st.dataframe(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

# # =========================== 🖼️ CNN Model for X-ray Image ============================= #
# st.header("🩻 Chest X-ray Diagnosis (CNN)")

# @st.cache_resource
# def load_cnn_model():
#     model = resnet18(pretrained=True)
#     model.fc = nn.Linear(model.fc.in_features, 2)
#     return model

# model = load_cnn_model()
# model.eval()

# image_file = st.file_uploader("🖼️ Upload Chest X-ray Image", type=['png', 'jpg', 'jpeg'])
# if image_file:
#     image = Image.open(image_file).convert('RGB')
#     st.image(image, caption="Uploaded Chest X-ray", width=250)

#     transform = transforms.Compose([
#         transforms.Resize((124, 224)),
#         transforms.ToTensor(),
#     ])
#     input_tensor = transform(image).unsqueeze(0)
#     output = model(input_tensor)
#     pred_label = torch.argmax(output, 1).item()
#     st.success("✅ Prediction: " + ("Normal" if pred_label == 0 else "⚠️ Abnormal"))

# # =========================== 🧪 Try with Custom Report ============================= #
# st.header("📝 Try with Custom Radiology Report")

# user_text = st.text_area("Enter a Radiology Report")
# if st.button("🔍 Analyze Report"):
#     if user_text.strip() != "":
#         input_vector = tfidf.transform([user_text])
#         pred = clf.predict(input_vector)[0]
#         st.success("✅ Report Prediction: " + ("Normal" if pred == 0 else "⚠️ Abnormal"))
#     else:
#         st.warning("⚠️ Please enter a report.")

# # =========================== 📤 Export Predictions ============================= #
# if uploaded_csv and st.button("📥 Export CSV with Predictions"):
#     df['predicted_label'] = clf.predict(tfidf.transform(df['report_text']))
#     df.to_csv("predicted_reports.csv", index=False)
#     st.success("✅ Exported as 'predicted_reports.csv'")




In [13]:

# ################ 4th 
# # ✅ All-in-One AI Medical Diagnosis (X-ray + Report Analysis + EDA + Metrics)

# import streamlit as st
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torchvision.transforms as transforms
# from torchvision.models import resnet18
# from PIL import Image
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os

# st.set_page_config(page_title="🧠 AI Medical Diagnosis", layout="wide")
# st.title("🩺 AI-Based Medical Diagnosis (Chest X-ray + Report NLP)")

# # =========================== 📥 Upload CSV and Show Preview ============================= #
# uploaded_csv = st.file_uploader("📄 Upload Radiology Report CSV", type="csv")
# if uploaded_csv:
#     df = pd.read_csv(uploaded_csv)
#     st.subheader("📊 Sample Data")
#     st.dataframe(df.head(), use_container_width=True)

#     st.subheader("📈 Label Distribution")
#     fig, ax = plt.subplots(figsize=(3, 2))
#     df['label'].value_counts().plot(kind='bar', ax=ax)
#     ax.set_xlabel("Label (0=Normal, 1=Abnormal)")
#     ax.set_ylabel("Count")
#     st.pyplot(fig)

#     # ======================== 📊 EDA and Column Profiling ============================= #
#     st.subheader("📌 Column-wise Info")
#     st.dataframe(df.describe(include='all'))

#     # =========================== 🧠 NLP - Report Text Classification ============================= #
#     st.subheader("🧠 Radiology Report Classifier (NLP)")
#     tfidf = TfidfVectorizer(max_features=300)
#     X = tfidf.fit_transform(df['report_text'])
#     y = df['label']

#     from sklearn.model_selection import train_test_split
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     clf = LogisticRegression()
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     acc = accuracy_score(y_test, y_pred)
#     st.write("✅ Accuracy:", round(acc * 100, 2), "%")

#     # Confusion Matrix (Smaller)
#     cm = confusion_matrix(y_test, y_pred)
#     st.subheader("📉 Confusion Matrix")
#     fig_cm, ax_cm = plt.subplots(figsize=(3, 2))
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax_cm, cbar=False)
#     ax_cm.set_xlabel("Predicted")
#     ax_cm.set_ylabel("Actual")
#     st.pyplot(fig_cm)

#     # Classification Report
#     st.subheader("🧾 Classification Report")
#     st.dataframe(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

# # =========================== 🖼️ CNN Model for X-ray Image ============================= #
# st.header("🩻 Chest X-ray Diagnosis (CNN)")

# @st.cache_resource
# def load_cnn_model():
#     model = resnet18(pretrained=True)
#     model.fc = nn.Linear(model.fc.in_features, 2)
#     return model

# model = load_cnn_model()
# model.eval()

# image_file = st.file_uploader("🖼️ Upload Chest X-ray Image", type=['png', 'jpg', 'jpeg'])
# if image_file:
#     image = Image.open(image_file).convert('RGB')
#     st.image(image, caption="Uploaded Chest X-ray", width=250)

#     transform = transforms.Compose([
#         transforms.Resize((124, 224)),
#         transforms.ToTensor(),
#     ])
#     input_tensor = transform(image).unsqueeze(0)
#     output = model(input_tensor)
#     pred_label = torch.argmax(output, 1).item()
#     st.success("✅ Prediction: " + ("Normal" if pred_label == 0 else "⚠️ Abnormal"))

# # =========================== 🧪 Try with Custom Report ============================= #
# st.header("📝 Try with Custom Radiology Report")

# user_text = st.text_area("Enter a Radiology Report")
# if st.button("🔍 Analyze Report"):
#     if user_text.strip() != "":
#         input_vector = tfidf.transform([user_text])
#         pred = clf.predict(input_vector)[0]
#         st.success("✅ Report Prediction: " + ("Normal" if pred == 0 else "⚠️ Abnormal"))
#     else:
#         st.warning("⚠️ Please enter a report.")

# # =========================== 📤 Export Predictions ============================= #
# if uploaded_csv and st.button("📥 Export CSV with Predictions"):
#     df['predicted_label'] = clf.predict(tfidf.transform(df['report_text']))
#     df.to_csv("predicted_reports.csv", index=False)
#     st.success("✅ Exported as 'predicted_reports.csv'")


In [14]:
# ############################# 2nd

# # ✅ All-in-One AI Medical Diagnosis (X-ray + Report Analysis)

# import streamlit as st
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torchvision.transforms as transforms
# from torchvision.models import resnet18
# from PIL import Image
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os

# st.set_page_config(page_title="🧠 AI Medical Diagnosis", layout="wide")
# st.title("🩺 AI-Based Medical Diagnosis (Chest X-ray + Report NLP)")

# # =========================== 📥 Upload CSV and Show Preview ============================= #
# uploaded_csv = st.file_uploader("📄 Upload Radiology Report CSV", type="csv")
# if uploaded_csv:
#     df = pd.read_csv(uploaded_csv)
#     st.subheader("📊 Sample Data")
#     st.write(df.head())

#     st.subheader("📈 Label Distribution")
#     fig, ax = plt.subplots()
#     df['label'].value_counts().plot(kind='bar', ax=ax)
#     ax.set_xlabel("Label (0=Normal, 1=Abnormal)")
#     st.pyplot(fig)

#     # ======================== 📊 EDA and Column Profiling ============================= #
#     st.subheader("📌 Column-wise Info")
#     st.write(df.describe(include='all'))

#     # =========================== 🧠 NLP - Report Text Classification ============================= #
#     st.subheader("🧠 Radiology Report Classifier (NLP)")
#     tfidf = TfidfVectorizer(max_features=300)
#     X = tfidf.fit_transform(df['report_text'])
#     y = df['label']
#     clf = LogisticRegression()
#     clf.fit(X, y)
#     y_pred = clf.predict(X)
#     st.write("✅ Accuracy:", accuracy_score(y, y_pred))

#     # Confusion Matrix
#     cm = confusion_matrix(y, y_pred)
#     st.write("📉 Confusion Matrix")
#     fig_cm, ax_cm = plt.subplots()
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax_cm)
#     ax_cm.set_xlabel("Predicted")
#     ax_cm.set_ylabel("Actual")
#     st.pyplot(fig_cm)

#     # Classification Report
#     st.write("🧾 Classification Report")
#     st.code(classification_report(y, y_pred), language="text")

# # =========================== 🖼️ CNN Model for X-ray Image ============================= #
# st.header("🩻 Chest X-ray Diagnosis (CNN)")

# @st.cache_resource
# def load_cnn_model():
#     model = resnet18(pretrained=True)
#     model.fc = nn.Linear(model.fc.in_features, 2)
#     return model

# model = load_cnn_model()
# model.eval()

# image_file = st.file_uploader("🖼️ Upload Chest X-ray Image", type=['png', 'jpg', 'jpeg'])
# if image_file:
#     image = Image.open(image_file).convert('RGB')
#     st.image(image, caption="Uploaded Chest X-ray", width=300)

#     transform = transforms.Compose([
#         transforms.Resize((124, 224)),
#         transforms.ToTensor(),
#     ])
#     input_tensor = transform(image).unsqueeze(0)
#     output = model(input_tensor)
#     pred_label = torch.argmax(output, 1).item()
#     st.success("✅ Prediction: " + ("Normal" if pred_label == 0 else "⚠️ Abnormal"))

# # =========================== 🧪 Try with Custom Report ============================= #
# st.header("📝 Try with Custom Radiology Report")

# user_text = st.text_area("Enter a Radiology Report")
# if st.button("🔍 Analyze Report"):
#     input_vector = tfidf.transform([user_text])
#     pred = clf.predict(input_vector)[0]
#     st.success("✅ Report Prediction: " + ("Normal" if pred == 0 else "⚠️ Abnormal"))

# # =========================== 📤 Export Predictions ============================= #
# if uploaded_csv and st.button("📥 Export CSV with Predictions"):
#     df['predicted_label'] = clf.predict(X)
#     df.to_csv("predicted_reports.csv", index=False)
#     st.success("✅ Exported as 'predicted_reports.csv'")




In [15]:

############################################################ old ################################3

# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# import numpy as np
# import cv2
# import os

# # Load and preprocess X-ray images
# image_path = "data/xray_images/"
# images, labels = [], []

# for file in os.listdir(image_path):
#     img = cv2.imread(os.path.join(image_path, file), cv2.IMREAD_GRAYSCALE)
#     img = cv2.resize(img, (224, 224))
#     images.append(img)
#     labels.append(0 if "normal" in file else 1)  # 0: Normal, 1: Pneumonia

# images = np.array(images) / 255.0  # Normalize
# images = images.reshape(-1, 224, 224, 1)
# labels = np.array(labels)

# # Define CNN model
# model = Sequential([
#     Conv2D(32, (3,3), activation='relu', input_shape=(224,224,1)),
#     MaxPooling2D(2,2),
#     Conv2D(64, (3,3), activation='relu'),
#     MaxPooling2D(2,2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid')
# ])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(images, labels, epochs=10, validation_split=0.2)
# model.save("models/xray_model.h5")


In [16]:
# from transformers import pipeline

# # Use a pre-trained BERT model
# classifier = pipeline("text-classification", model="bert-base-uncased")

# def predict_medical_text(text):
#     result = classifier(text)
#     return result[0]['label']

# # Example Usage
# sample_text = "Patient has severe lung infection with pneumonia symptoms."
# print(predict_medical_text(sample_text))


In [17]:
# from flask import Flask, request, jsonify
# import tensorflow as tf
# import cv2
# import numpy as np
# from transformers import pipeline

# app = Flask(__name__)

# # Load CNN Model
# cnn_model = tf.keras.models.load_model("models/xray_model.h5")

# # Load NLP Model
# nlp_classifier = pipeline("text-classification", model="bert-base-uncased")

# @app.route('/predict/xray', methods=['POST'])
# def predict_xray():
#     file = request.files['file']
#     img = cv2.imdecode(np.fromstring(file.read(), np.uint8), cv2.IMREAD_GRAYSCALE)
#     img = cv2.resize(img, (224, 224)) / 255.0
#     img = img.reshape(1, 224, 224, 1)

#     prediction = cnn_model.predict(img)[0][0]
#     result = "Pneumonia" if prediction > 0.5 else "Normal"
#     return jsonify({"diagnosis": result})

# @app.route('/predict/text', methods=['POST'])
# def predict_text():
#     text = request.json['text']
#     result = nlp_classifier(text)
#     return jsonify({"diagnosis": result[0]['label']})

# if __name__ == '__main__':
#     app.run(debug=True)


In [18]:
# import streamlit as st
# import requests
# from PIL import Image

# st.title("🩺 AI-Based Medical Diagnosis System")

# # Upload X-ray Image
# st.header("📷 Upload Chest X-ray")
# uploaded_file = st.file_uploader("Choose an X-ray image...", type=["jpg", "png"])

# if uploaded_file is not None:
#     image = Image.open(uploaded_file)
#     st.image(image, caption="Uploaded Image", use_column_width=True)

#     # Send to API
#     files = {"file": uploaded_file.getvalue()}
#     response = requests.post("http://127.0.0.1:5000/predict/xray", files=files)
#     st.write("Diagnosis:", response.json()["diagnosis"])

# # Medical Text Input
# st.header("📝 Enter Medical Report")
# text_input = st.text_area("Enter medical description here...")

# if st.button("Analyze Text"):
#     response = requests.post("http://127.0.0.1:5000/predict/text", json={"text": text_input})
#     st.write("Diagnosis:", response.json()["diagnosis"])


In [19]:
# FROM python:3.8

# WORKDIR /app
# COPY requirements.txt .
# RUN pip install -r requirements.txt

# COPY . .
# CMD ["python", "api.py"]


In [20]:
# 🚀 1. AI-Powered Resume Parser & Job Matcher
# 🔹 Tech Stack: Python, NLP (Spacy, BERT), ML (Scikit-learn), Deep Learning (Transformers), Flask, MongoDB
# 🔹 Project Scope:

#     Build an AI-powered system that extracts key information from resumes (Name, Skills, Experience) and matches candidates with job descriptions.

#     Use NLP techniques like Named Entity Recognition (NER) with BERT or SpaCy.

#     Implement TF-IDF and Cosine Similarity for job recommendations.

#     Deploy the model with Flask/FastAPI and a database like MongoDB for storing job data.

# 🔹 Why This is Advanced?

#     Requires BERT embeddings for resume parsing.

#     Combines NLP, ML, and AI in one project.

#     Real-world impact for HR and recruitment industries.

# 🚀 2. AI-Based Fake News Detection System
# 🔹 Tech Stack: Python, NLP (BERT, LSTMs), ML (Scikit-learn), Deep Learning (CNNs, RNNs), Flask
# 🔹 Project Scope:

#     Collect news articles and tweets from sources like Kaggle or Twitter API.

#     Preprocess text (stopword removal, stemming, TF-IDF).

#     Train an LSTM or CNN model for classification (Fake vs Real news).

#     Use BERT embeddings for improved accuracy.

#     Deploy it as a browser extension or web app for users to verify news credibility.

# 🔹 Why This is Advanced?

#     Uses transformers (BERT) for context understanding.

#     Combines Deep Learning (CNNs, RNNs) + NLP.

#     Real-world application to fight misinformation.

# 🚀 3. AI Chatbot with Speech Recognition and Emotional Intelligence
# 🔹 Tech Stack: Python, NLP (Transformer Models like GPT, DialogFlow), Deep Learning (CNN for Emotion Detection), Flask, React Native
# 🔹 Project Scope:

#     Train a chatbot using GPT-3.5 or LLaMA models.

#     Integrate Speech-to-Text and Text-to-Speech using OpenAI Whisper or Google Speech API.

#     Use CNNs and facial recognition to detect user emotions from webcam.

#     Make chatbot responses more empathetic based on user emotions.

#     Deploy as a mobile app using React Native.

# 🔹 Why This is Advanced?

#     Involves AI-driven chatbot with speech processing.

#     Combines NLP, CNN, and Deep Learning for emotion detection.

#     Advanced real-world AI for mental health and customer support.

# 🚀 4. AI-Based Medical Diagnosis System (X-ray & Text Analysis)
# 🔹 Tech Stack: Python, Deep Learning (CNNs, Transfer Learning), NLP (Medical Reports Analysis), Flask
# 🔹 Project Scope:

#     Train a CNN model on chest X-ray datasets to detect pneumonia, lung cancer, etc.

#     Apply NLP techniques to process radiology reports and extract useful insights.

#     Combine both image and text inputs to make predictions more accurate.

#     Deploy the model using Flask and build a web app for doctors.

# 🔹 Why This is Advanced?

#     Multi-modal learning (image + text processing together).

#     Uses CNNs for medical image classification.

#     Real-world impact in healthcare AI.

# 🚀 5. AI-Based Video Summarization & Captioning System
# 🔹 Tech Stack: Python, Deep Learning (LSTMs, CNNs, Transformers), NLP (BERT), Flask
# 🔹 Project Scope:

#     Build an AI model that automatically summarizes videos into short clips.

#     Use CNNs and LSTMs to analyze video frames and audio.

#     Generate automatic captions using Transformer models (BART/T5).

#     Deploy the AI model on a web platform for content creators.

# 🔹 Why This is Advanced?

#     Involves video processing (CNNs) + NLP-based text generation.

#     Uses Deep Learning models like LSTMs & Transformers.

#     Useful in YouTube, TikTok, and media platforms.

# 🎯 Which one do you want to start with?

# I can guide you step by step on implementing any of these projects! 🚀
# 2/2
# You said:
# give me which is most important in between 2, 4 and 5
# ChatGPT said:

# Between Fake News Detection (2), AI-Based Medical Diagnosis (4), and Video Summarization & Captioning (5), the most important project depends on your goals:
# 📌 If You Want High Impact & Job Relevance:
# ✅ AI-Based Medical Diagnosis System (4) - Most Important

#     Reason:

#         Healthcare AI is booming and has a massive real-world impact.

#         Multi-modal learning (combining text + images) is a cutting-edge research area.

#         Companies in health tech, AI in radiology, and medical NLP are actively hiring in this field.

#         It uses CNNs (for X-rays) + NLP (for medical text analysis), making it a very strong AI project.

# 📌 If You Want Real-World Application & NLP Experience:
# ✅ Fake News Detection (2) - Second Most Important

#     Reason:

#         Misinformation is a global problem, making this project valuable.

#         Uses Transformers (BERT), CNNs, and NLP, which are in high demand.

#         Applicable in journalism, cybersecurity, and AI research.

# 📌 If You Want an AI Project for Media & Content Industry:
# ✅ Video Summarization & Captioning (5) - Third Most Important

#     Reason:

#         Good for AI-driven media applications (YouTube, TikTok, Instagram, etc.).

#         Involves computer vision (CNNs) + NLP (Transformers, LSTMs) for automatic captioning.

#         Useful for video editing automation, content creators, and AI-powered video indexing.

# 🏆 Final Verdict (Ordered by Importance)

# 1️⃣ Medical Diagnosis AI (4) - Most Important (High impact in healthcare, multi-modal AI)
# 2️⃣ Fake News Detection (2) - Second (Valuable in journalism, misinformation detection)
# 3️⃣ Video Summarization & Captioning (5) - Third (Best for media, NLP & vision applications)

# If your goal is job opportunities, go with Medical AI (4).
# If you want to work with NLP, pick Fake News Detection (2).
# If you're interested in media & AI applications, pick Video Summarization (5).
# 👉 Which one do you want to start with? I can guide you step by step! 🚀
# You said:
# yes start 4
# ChatGPT said:

# Great choice! The AI-Based Medical Diagnosis System is a challenging yet impactful project. Here's how we can structure it:
# Project Overview

# This project will leverage CNNs for analyzing X-ray images to detect diseases and NLP for extracting insights from medical reports.
# Key Components

#     Data Collection

#         X-ray images (e.g., ChestX-ray14, COVID-19 dataset).

#         Medical reports from public datasets.

#     Image Processing & CNN

#         Preprocess images (resizing, normalization).

#         Train a CNN model (ResNet, EfficientNet) for disease classification.

#     Medical Text Processing with NLP

#         Preprocess medical text (tokenization, stop-word removal).

#         Train an NLP model (BERT, BioBERT) to classify and extract medical terms.

#     Model Integration

#         Combine CNN and NLP models to provide a comprehensive diagnosis.

#         Develop a Flask/FastAPI backend to serve predictions.

#     Deployment

#         Use AWS/GCP for hosting models.

#         Build a simple React frontend for user interaction.

# Would you like me to help with dataset selection or model architecture first? 🚀
# You said:
# yes and give me step by step procedure and also give me code of 
# this
# ChatGPT said:
# Step-by-Step Procedure for AI-Based Medical Diagnosis System

# We will develop an AI-Based Medical Diagnosis System that analyzes X-ray images using CNNs and processes medical text using NLP for enhanced diagnosis.
# Step 1: Project Setup

#     Install the required libraries:

#   pip install tensorflow keras numpy pandas matplotlib seaborn opencv-python nltk transformers scikit-learn


In [21]:
# Step 1: Project Setup

#     Install the required libraries:

# pip install tensorflow keras numpy pandas matplotlib seaborn opencv-python nltk transformers scikit-learn

# Create a structured project directory:

#     medical-diagnosis/
#     ├── data/               # Store X-ray images and medical text dataset
#     ├── models/             # Store trained CNN and NLP models
#     ├── src/                # Source code
#     ├── notebooks/          # Jupyter notebooks for exploration
#     ├── app/                # Web/App interface (optional)
#     ├── requirements.txt    # Dependencies list
#     ├── main.py             # Main script

# Step 2: Data Collection

#     Use open datasets like:

#         Chest X-ray Dataset: NIH Chest X-rays

#         Medical Text Dataset: MIMIC-III

#     Download and preprocess images:

# import cv2
# import os
# import numpy as np

# image_path = "data/xray_images/"
# images = []
# labels = []

# for file in os.listdir(image_path):
#     img = cv2.imread(os.path.join(image_path, file), cv2.IMREAD_GRAYSCALE)
#     img = cv2.resize(img, (224, 224))
#     images.append(img)
#     labels.append(file.split("_")[0])  # Assuming labels in file names

# images = np.array(images) / 255.0  # Normalize

# Process medical text:

#     import pandas as pd
#     from nltk.tokenize import word_tokenize

#     text_data = pd.read_csv("data/medical_reports.csv")
#     text_data["tokens"] = text_data["report"].apply(word_tokenize)

# Step 3: Train CNN Model for X-ray Diagnosis

# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# # CNN Model
# model = Sequential([
#     Conv2D(32, (3,3), activation='relu', input_shape=(224,224,1)),
#     MaxPooling2D(2,2),
#     Conv2D(64, (3,3), activation='relu'),
#     MaxPooling2D(2,2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid')  # Binary classification
# ])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(images, labels, epochs=10, validation_split=0.2)
# model.save("models/xray_model.h5")

# Step 4: Train NLP Model for Medical Report Analysis

# from transformers import pipeline

# # Use a pre-trained BERT model for medical text classification
# classifier = pipeline("text-classification", model="bert-base-uncased")

# sample_text = "Patient has pneumonia with visible infection in the lungs."
# result = classifier(sample_text)
# print(result)

# Step 5: Deploying the System

#     Use Flask or FastAPI to build an API:

# from flask import Flask, request, jsonify
# import tensorflow as tf
# import cv2
# import numpy as np

# app = Flask(__name__)
# model = tf.keras.models.load_model("models/xray_model.h5")

# @app.route('/predict', methods=['POST'])
# def predict():
#     file = request.files['file']
#     img = cv2.imdecode(np.fromstring(file.read(), np.uint8), cv2.IMREAD_GRAYSCALE)
#     img = cv2.resize(img, (224,224)) / 255.0
#     img = img.reshape(1, 224, 224, 1)
    
#     prediction = model.predict(img)
#     return jsonify({"diagnosis": "Positive" if prediction[0][0] > 0.5 else "Negative"})

# if __name__ == '__main__':
#     app.run(debug=True)

# Step 6: Frontend Development

#     Use React.js or Streamlit for a simple UI.

# Step 7: Deployment on Cloud

#     Deploy the API on AWS, GCP, or Heroku.

#     Use Docker and Kubernetes for scaling.

# Next Steps

#     Improve the CNN model with ResNet or EfficientNet.

#     Use BERT fine-tuning for NLP accuracy.

#     Add Explainable AI (XAI) for better interpretability.

# Would you like any specific part to be improved or explained further? 🚀
